In [1]:
from pynq import Overlay
# load overlay from bitstream
ovl = Overlay("rx_ctrl.bit")

/root/venv/lib/python3.8/site-packages/pynq/pl_server/xrt_device.py:59: UserWarning: xbutil failed to run - unable to determine XRT version
  warnings.warn("xbutil failed to run - unable to determine XRT version")


In [ ]:
# peak overlay info
ovl.ip_dict

In [3]:
# aliasing kernel
rx_krnl = ovl.rx_ctrl_0

In [4]:
# checkout control signals
rx_krnl.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, AP_CONTINUE=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  store_DRAM_1 = Register(store_DRAM=write-only),
  store_DRAM_2 = Register(store_DRAM=write-only)
}

In [5]:
# prepare output buffer
from pynq import allocate
import numpy as np

output_size = 1100
output_buffer = allocate(shape=(output_size,), dtype=np.int32)

In [6]:
# set address for hls kernel
output_buffer_addr = output_buffer.physical_address
rx_krnl.write(0x10, output_buffer_addr)

In [7]:
# call kernel by writing to control register
CONTROL_REGISTER = 0x00
rx_krnl.write(CONTROL_REGISTER, 1) # 0x81 will set bit 0

In [9]:
# checkout control signals
rx_krnl.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=1, AP_IDLE=1, AP_READY=0, AP_CONTINUE=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  store_DRAM_1 = Register(store_DRAM=write-only),
  store_DRAM_2 = Register(store_DRAM=write-only)
}

In [10]:
sum=0
for i in range(1000):
    # print(output_buffer[i])
    sum+=output_buffer[i]
print(sum)

500


In [11]:
# checkout control signals
rx_krnl.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=1, AP_IDLE=1, AP_READY=0, AP_CONTINUE=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  store_DRAM_1 = Register(store_DRAM=write-only),
  store_DRAM_2 = Register(store_DRAM=write-only)
}

In [12]:
for i in range(999):
    if output_buffer[i+1] - output_buffer[i] != -1:
        print("mismatch: {}, {} at {}".format(output_buffer[i+1], output_buffer[i], i))

In [ ]:
for i in range(1000):
    print(output_buffer[i])